In [39]:
import matplotlib.pyplot as plt
import numpy as np
from keras.datasets import mnist
from keras.utils.np_utils import to_categorical
np.random.seed(1234)
import os
import sys
import tensorflow as tf
import time
tf.random.set_seed(1234)


In [40]:
#(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
(X_train, y_train), (X_test, y_test) = mnist.load_data()



X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

In [41]:
print('Input dimensions')
print (X_train.shape, X_test.shape)
print (y_train.shape, y_test.shape)

X_train = X_train.reshape(60000, 28*28)
X_test = X_test.reshape(10000, 28*28)

print ('After reshaping:', X_train.shape, X_test.shape)

Input dimensions
(60000, 28, 28) (10000, 28, 28)
(60000,) (10000,)
After reshaping: (60000, 784) (10000, 784)


In [42]:
nb_classes = 10
y_train = to_categorical(y_train, nb_classes)
y_test = to_categorical(y_test, nb_classes)

In [43]:
# import of sigmoid and crossentropy
from scipy.special import expit
from scipy.special import softmax
from sklearn.metrics import log_loss

In [82]:
class DFA:
  def __init__(self):
    self.W1 = np.random.randn(800, 784)
    self.W2 = np.random.randn(10, 800)
    self.b1 = np.zeros((800, 1)) 
    self.b2 = np.zeros((10, 1))
  def forward_pass(self, x):
    '''This is the forward pass. It is equal for any
    training algorithm. It's just one hidden layer
    with tanh activation function and sigmoid on the
    output layer'''
    # if the input is a batch, I have to tile as many
    # b1 and b2 as the batch size
    a1 = np.matmul(self.W1, x)+np.tile(self.b1, x.shape[1])
    h1 = np.tanh(a1)
    a2 = np.matmul(self.W2, h1)+np.tile(self.b2, x.shape[1])
    y_hat = softmax(a2)
    return a1, h1, a2, y_hat

  def dfa_backward_pass(self, e, h1, B1, a1, x):
    dW2 = -np.matmul(e, np.transpose(h1))
    da1 = np.matmul(B1, e)*(1-np.tanh(a1)**2)
    dW1 = -np.matmul(da1, np.transpose(x))
    db1 = -np.sum(da1, axis=1)
    db2 = -np.sum(e, axis=1)
    return dW1, dW2, db1[:,np.newaxis], db2[:,np.newaxis]
  def average_angle(self, B1, error, a1, a2):
    dh1 = np.mean(np.matmul(B1, error), axis=1)[:, np.newaxis] #forse non ci va la derivata
    c1 = np.mean(np.matmul(np.transpose(self.W2), error*(expit(a2)*(1-expit(a2)))), axis=1)[:, np.newaxis]
    dh1_norm = np.linalg.norm(dh1)
    c1_norm = np.linalg.norm(c1)
    inverse_dh1_norm = np.power(dh1_norm, -1)
    inverse_c1_norm = np.power(c1_norm, -1)
    
    # ALIGNMENT CRITERION AND ANGLE
    Lk = (np.matmul(np.transpose(dh1), c1)*inverse_dh1_norm)[0, 0]
    beta = np.arccos(np.clip(Lk*inverse_c1_norm, -1., 1.))*180/np.pi
    return Lk, beta
  def dfa_train(self,x, y, n_epochs=10, lr=1e-3, batch_size=200, tol=1e-3):
    x = np.transpose(x)
    y = np.transpose(y)
    
    B1 = np.random.randn(800, 10)
    dataset_size = x.shape[1]
    n_batches = dataset_size//batch_size
    te_dfa = []
    angles = []
    for i in range(n_epochs):
        perm = np.random.permutation(x.shape[1])
        x = x[:, perm]
        y = y[:, perm]
        loss = 0.
        train_error = 0.
        for j in range(n_batches):
            samples = x[:, j*batch_size:(j+1)*batch_size]
            targets = y[:, j*batch_size:(j+1)*batch_size]
            a1, h1, a2, y_hat = self.forward_pass(samples)
            error = y_hat - targets
            preds = np.argmax(y_hat, axis=0) 
            truth = np.argmax(targets, axis=0)
            train_error += 1.*np.sum(preds!=truth)
            loss_on_batch = log_loss(targets, y_hat)
            
            dW1, dW2, db1, db2 = self.dfa_backward_pass(error, h1, B1, a1, samples)
            self.W1 += lr*dW1
            self.W2 += lr*dW2
            self.b1 += lr*db1
            self.b2 += lr*db2
            loss += loss_on_batch
            if j%100==0:
                angles.append(self.average_angle(B1, error, a1, a2))
        training_error = 1.*train_error/x.shape[1]
        a1, h1, a2, y_hat = self.forward_pass(samples)
        #print(y_hat.shape)
        #print(targets.shape)
        #break
        accur=self.accuracy_function(y_hat,targets)
        accur = accur * 100
        bw = x.shape[1]/n_batches
        print("Training Accuracy = {}".format(accur.numpy()))
        print('Loss at epoch', i+1, ':', loss/x.shape[1])
        print('Training error:', training_error)
        prev_training_error = 0 if i==0 else te_dfa[-1]
        if np.abs(training_error-prev_training_error) <= tol:
            te_dfa.append(training_error)
            break
        te_dfa.append(training_error)
    return te_dfa, angles
  def test(self, test_samples, test_targets):
    test_samples = np.transpose(test_samples)
    test_targets = np.transpose(test_targets)
    outs = self.forward_pass(test_samples)[-1]
    preds = np.argmax(outs, axis=0) 
    truth = np.argmax(test_targets, axis=0)
    test_error = 1.*np.sum(preds!=truth)/preds.shape[0]
    return test_error
  def accuracy_function(self, yhat,true_y):
    #yhat = tf.nn.softmax(yhat)
    yhat = yhat
    preds = np.argmax(yhat, axis=0) 
    truth = np.argmax(true_y, axis=0)
    correct_prediction = tf.equal(preds, true_y)
    #correct_prediction = tf.equal(tf.argmax(yhat, 1), tf.argmax(true_y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy


In [83]:
model=DFA()
te_dfa, angles = model.dfa_train(X_train, y_train, n_epochs=30, lr=1e-3, batch_size=200, tol=1e-3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in power
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in power
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars


Training Accuracy = 47.54999923706055
Loss at epoch 1 : 0.953074333413373
Training error: 0.3961


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in power
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in power
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in power
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars


Training Accuracy = 79.79999542236328
Loss at epoch 2 : 0.8309395622028537
Training error: 0.6637166666666666


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in power
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in power
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in power
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars


...

Training Accuracy = 87.5999984741211
Loss at epoch 11 : 0.8389457057315378
Training error: 0.8563166666666666


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in power
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in power
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in power
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars


Training Accuracy = 88.4000015258789
Loss at epoch 12 : 0.8396360783838531
Training error: 0.8599666666666667


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in power
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in power
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in power
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars


Training Accuracy = 84.4000015258789
Loss at epoch 13 : 0.8394292738101781
Training error: 0.8600166666666667


In [84]:
print ('DFA:', model.test(X_test, y_test)*100, '%')


DFA: 88.14 %


In [85]:
!pip install torchattacks

In [86]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision.utils
from torchvision import models
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torchattacks
from torchattacks import PGD, FGSM

In [87]:
class MLP2(nn.Module):
  '''
    Multilayer Perceptron.
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Flatten(),
      nn.Linear(784, 256),
      nn.ReLU(),
      nn.Linear(256, 256),
      nn.ReLU(),
      nn.Linear(256, 256),
      nn.ReLU(),
      nn.Linear(256, 10)
    )


  def forward(self, x):
    '''Forward pass'''
    return self.layers(x)

In [88]:
mnist_train = dsets.MNIST(root='./data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='./data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [89]:
batch_size = 50

train_loader  = torch.utils.data.DataLoader(dataset=mnist_train,
                                           batch_size=batch_size,
                                           shuffle=False)
#change here 
test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                         batch_size=10000,
                                         shuffle=False)

In [90]:
model2 = MLP2().cpu()

In [91]:
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model2.parameters(), lr=0.001)

In [92]:
atk = PGD(model2, eps=0.3, alpha=0.1, steps=7)
num_epochs = 5
for epoch in range(num_epochs):

    total_batch = len(mnist_train) // batch_size
    
    for i, (batch_images, batch_labels) in enumerate(train_loader):
        X = atk(batch_images, batch_labels).cpu()
        Y = batch_labels.cpu()

        pre = model2(X)
        cost = loss(pre, Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print('Epoch [%d/%d], lter [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, total_batch, cost.item()))

Epoch [1/5], lter [100/1200], Loss: 2.2888
Epoch [1/5], lter [200/1200], Loss: 2.2686
Epoch [1/5], lter [300/1200], Loss: 2.1839
Epoch [1/5], lter [400/1200], Loss: 2.2428
Epoch [1/5], lter [500/1200], Loss: 2.1846
Epoch [1/5], lter [600/1200], Loss: 2.2274
Epoch [1/5], lter [700/1200], Loss: 2.1311
Epoch [1/5], lter [800/1200], Loss: 2.1348
Epoch [1/5], lter [900/1200], Loss: 2.1460
Epoch [1/5], lter [1000/1200], Loss: 2.1473
Epoch [1/5], lter [1100/1200], Loss: 2.0695
Epoch [1/5], lter [1200/1200], Loss: 2.1906
Epoch [2/5], lter [100/1200], Loss: 2.0061
Epoch [2/5], lter [200/1200], Loss: 2.0630
Epoch [2/5], lter [300/1200], Loss: 1.9767
Epoch [2/5], lter [400/1200], Loss: 2.1179
Epoch [2/5], lter [500/1200], Loss: 2.0696
Epoch [2/5], lter [600/1200], Loss: 2.1971
Epoch [2/5], lter [700/1200], Loss: 1.9244
Epoch [2/5], lter [800/1200], Loss: 1.9815
Epoch [2/5], lter [900/1200], Loss: 1.9935
Epoch [2/5], lter [1000/1200], Loss: 2.0348
Epoch [2/5], lter [1100/1200], Loss: 1.9548
Epoch 

In [93]:
model2.eval()

correct = 0
total = 0

atk = FGSM(model2, eps=0.3)
iter = 0
accuracy_test = 0.0

img = []
lbl = []
for images, labels in test_loader:
    #80/20 split and make sure labels are same 
    if iter == 0:
      images = atk(images, labels).cpu()
   
    images = images.reshape(images.shape[0], 28*28)
    images = tf.cast(images, tf.float32)
    img.append(images)
    label1 = tf.one_hot(labels,depth=10)
    label1 = tf.cast(label1, tf.float32)
    lbl.append(label1)
    iter +=1

img_test = tf.concat(img,0)
lbl_test = tf.concat(lbl,0)
img_test= np.transpose(img_test)
lbl_test= np.transpose(lbl_test)

a1, h1, a2, y_hat = model.forward_pass(img_test)
accur=model.accuracy_function(y_hat,lbl_test)
accur = accur * 100
print("Training Accuracy = {}".format(accur.numpy()))


Training Accuracy = 89.9729995727539
